In [3]:
import pandas as pd
import ast

# Load the recipes file
recipes_df = pd.read_csv('recipes_with_cuisine.csv')

# Load the ingredients labels file
labels_df = pd.read_csv('labels/ingredients.csv')

# Convert ingredients from string representation to actual lists
recipes_df['ingredients'] = recipes_df['ingredients'].apply(ast.literal_eval)

# Create a dictionary for ingredient synonyms and their categories
ingredient_categories = {}
for _, row in labels_df.iterrows():
    synonyms = row['Ingredient Synonyms'].split('; ')
    for synonym in synonyms:
        ingredient_categories[synonym.lower()] = row['Category']
    # Add the aliased ingredient name as well
    ingredient_categories[row['Aliased Ingredient Name'].lower()] = row['Category']

# Function to categorize ingredients in a recipe
def categorize_ingredients(ingredient_list):
    categories = set()
    for ingredient in ingredient_list:
        # Normalize the ingredient to lowercase for matching
        normalized_ingredient = ingredient.lower()
        for key in ingredient_categories:
            # Skip if the category or key is 'dish'
            if key == 'dish' or key == 'cereal':
                continue
            if key in normalized_ingredient:
                category = ingredient_categories[key]
                if category.lower() != 'dish' and category.lower() != 'cereal':
                    categories.add(category)
    return list(categories)

# Apply categorization to each recipe's ingredients
recipes_df['categories'] = recipes_df['ingredients'].apply(categorize_ingredients)

# Save the updated DataFrame to a new CSV file
recipes_df.to_csv('recipes_with_unique_categories.csv', index=False)

# Display a sample of the updated DataFrame
print(recipes_df[['title', 'ingredients', 'categories']].head())

                   title                                        ingredients  \
0    no-bake nut cookies  [1 c. firmly packed brown sugar, 1/2 c. evapor...   
1  jewell ball's chicken  [1 small jar chipped beef, cut up, 4 boned chi...   
2            creamy corn  [2 (16 oz.) pkg. frozen corn, 1 (8 oz.) pkg. c...   
3          chicken funny  [1 large whole chicken, 2 (10 1/2 oz.) cans ch...   
4   reeses cups(candy)    [1 c. peanut butter, 3/4 c. graham cracker cru...   

                                          categories  
0  [Essential Oil, Bakery, Dairy, Fruit, Additive...  
1                              [Dairy, Meat, Fungus]  
2              [Herb, Maize, Spice, Dairy, Additive]  
3                    [Dairy, Meat, Additive, Fungus]  
4  [Beverage Alcoholic, Meat, Legume, Dairy, Plan...  
